## Some introductory notes
For our labs, we use Jupyter notebooks. These are increasingly being used in present-day data science projects. As advantages, rather than having to write scripts or using command prompts, you can interactively work and code in your browser, 'play around' with parameters and settings while keeping an overview of what steps you took, and e.g. include surrounding markdown descriptions and plots.

As not everyone of you may be fluent in Python, our labs are constructed such that you will not have to do any setups from scratch, and you don't need in-depth program-linguistic knowledge up front. Instead, we will rather offer boilerplate code and nearly-completed code, and you will be asked to run this, complete this and adapt this.

Throughout the notebook, you also will find several <b>Questions</b>. These are meant for you to discuss in your duo while conducting the lab. Note you do <b>not</b> have to submit the answers to these questions to us as part of the final deliverable: they are purely meant for reflection.

To execute code in a notebook cell, hit the 'execute' button in the browser or use the Ctrl-Enter shortcut.

# Lab 1: Recommender systems, Colab version

Following the second lecture of this week, for this lab, we will zoom in into the workings of recommender systems.

More specifically, we are going to try out matrix factorization on a slice of the first Movielens dataset.

While doing this, you also will be introduced to a few handy data handling routines in Python.

## About the Movielens data
Movielens (https://movielens.org/) is a movie recommendation service run by the Grouplens (http://grouplens.org/) team at the University of Minnesota. Over the years, the service has been growing considerably, and once every few years, Grouplens releases new anonymized usage data to the community (http://grouplens.org/datasets/movielens/), which have become de facto benchmarking standards in the Recommender Systems research community.

In 1998, the first released dataset consisted of 100,000 ratings (1000 users, 1700 movies). The most recently released benchmark dataset (2014) consisted of 20 million ratings (138,000 users, 27,000 movies).

In the current lab session, we will not focus on the big data crunching, but rather on getting a better practical feeling on how to handle user-item data, as well as the implications of Matrix Factorization.

## What to hand in
As final deliverable to demonstrate your successful completion of this assignment, please submit a file named [studentNumberMember1_studentNumberMember2.pdf] through Brightspace.

This file should:
* Include **three** latent factor scatter plots, obtained for **three different input data slices** (which should differ at least in terms of user population size).
* For each of the scatter plots, please:
  * specify the characteristics of your input data (e.g. population size, selection criteria for users and films);
  * based on the plot, give a possible interpretation of what the factors represented by the x and y axes of your plot may indicate.

Further instructions can be found further down this notebook, at the point where we give an example plot.

## Installing dependencies to run on Google Colab!
We will first setup the system and install the dependencies for this lab

**Note**: This cell needs to be run everytime you start a new session of the Colab Notebook

In [0]:
import os, time


!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Packages for the lab notebooks
!apt-get -qq -y install mono-complete
!apt-get -qq -y install libsamplerate0 libsamplerate0-dev

!pip install --upgrade setuptools

### One-time user authentication for accessing google drive folders

 - Open the link that comes in the output cell and copy & paste the token in the text box here.
 - If just the text box appears (i.e textbox without authentication links), then input your google password in the textbox.

**Note**: This cell needs to be run everytime you start a new session of the Colab Notebook

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

### Accessing Google Drive files

In this step, we basically create an empty directory and then **mount** `google-drive`(google drive directory) on the empty directory that we have created.

In [0]:
# Check the current working directory first
!pwd
!ls

In [0]:
# If the gdrive folder does not exist already then perform the following steps, else skip this
!mkdir -p gdrive
!google-drive-ocamlfuse gdrive
# !google-drive-ocamlfuse nonempty gdrive

### Change working directory to the MMSR repository folder

In [0]:
# Check the current working directory
!pwd
!ls 

#### If running for the first time, **`CREATE`** a new directory for the MMSR labs.
**NOTE**: If the directory has already been created, then skip this cell

In [0]:
os.mkdir("gdrive/MMSR_lab")

#### Change to the MMSR lab directory

In [0]:
# Change to the created directory
os.chdir("gdrive/MMSR_lab")

# Verify if we are in the right directory
!pwd

### Clone the MMSR repository (https://gitlab.ewi.tudelft.nl/mmc-tudelft/education/cs4065.git)
Alternatively, you can download the MMSR repository on your local machine and then upload the repository folder manually to the `MMSR_lab` folder.

**NOTE**: Skip this cell if the repository is already cloned or uploaded manually

In [0]:
!git clone https://gitlab.ewi.tudelft.nl/mmc-tudelft/education/cs4065.git

**Important**: Check if the repository exists in the current directory

In [0]:
!ls

In [0]:
# Change directory to the repository
os.chdir("cs4065")
!ls

### Install python dependencies for this lab exercise

Python requirements for the **Colab Notebook** versions can be found in the *colab_requirements* directory

For this lab, run `lab2-requirements.txt`

In [0]:
!pip install -r colab_requirements/lab1-requirements.txt

## Importing the data
We have prepared a subset of the Movielens 100k dataset, which should automagically be downloaded to your (virtual) machine:

In [0]:
from datasets import CS4065_Dataset
DATASET_DIR = os.path.join(os.getcwd(), "data")
print(DATASET_DIR)


# Load dataset of images.
cs4065_data = CS4065_Dataset(store_dir=DATASET_DIR)
movielens_paths = cs4065_data.get_movielens_subset()

<code>movielens_paths</code> is a dictionary, keyed by three file names in the Movielens 100k dataset:
* <code>u.data</code>, containing tab-separated rating data (specifying user ID, movie ID, rating, timestamp);
* <code>u.user</code>, containing pipe-separated anonymized demographics data for all users in the dataset (specifying user ID, age, gender, occupation, ZIP code);
* <code>u.item</code>, containing pipe-separated movie background information (specifying movie ID, title, genres, IMDB URLs, and more).

In [0]:
# Verify the contents of movielens_paths
print(movielens_paths)

## Selecting analysis data

In this lab, we will primarily use data from <code>u.item</code>. The information in <code>u.user</code> is interesting though for assessing potential characterizing user features, and the information in <code>u.item</code> was used by us to manually select a small set of movie IDs, for which we expect that the fan base will show some variation.

We put our selection in a <code>movie_data</code> dictionary:

In [0]:
# Initialization of the dictionary.
# It will be keyed by movie ID, and have the IMDB movie title as corresponding value.
movie_data = {}

# We now populate the dictionary with some manually chosen examples,
# for which we expect to see some different underlying fan bases.

# First, we add three animated movies about princesses.
movie_data[418] = 'Cinderella (1950)'
movie_data[538] = 'Anastasia (1997)'
movie_data[542] = 'Pocahontas (1995)'
# Then, we add three psychological horror movies.
movie_data[200] = 'Shining, The (1980)'
movie_data[98] = 'Silence of the Lambs, The (1991)'
movie_data[185] = 'Psycho (1960)'
# Subsequently, we add three musical movies.
movie_data[186] = 'Blues Brothers, The (1980)'
movie_data[451] = 'Grease (1978)'
movie_data[289] = 'Evita (1996)'
# And finally, we add three movies taking place in space.
movie_data[50] = 'Star Wars (1977)'
movie_data[89] = 'Blade Runner (1982)'
movie_data[135] = '2001: A Space Odyssey (1968)'

# Now, we have a dictionary with 12 items:
print(movie_data)
print('\nMovie %d is titled "%s".' % (200, movie_data[200]))

## Extracting the rating data

Next step: we won't look at 100,000 ratings, but **only at ratings for any of these movies**.

Let's first put the IDs of the movies we are interested in in a separate variable.

In [0]:
movies_considered = movie_data.keys()
# For convenience, we will sort the movies by ID.
movies_considered = sorted(movies_considered)
# How many movies will we consider again?
len(movies_considered)

For extracting the rating data, we will use the <code>pandas</code> module. It offers some more advanced data structures and corresponding handling capabilities than simple Python dictionaries would be capable of.

We will illustrate some of the possibilities here, but see http://pandas.pydata.org/ and the documentation at http://pandas.pydata.org/pandas-docs/version/0.17.1/ for more extensive background and more advanced examples.

In [0]:
import pandas

A first neat feature is that we can use the <code>read_csv</code> function to read formatted data into a so-called DataFrame:

In [0]:
# let's check the help documentation for read_csv
help(pandas.read_csv)

Clearly, there are many options.
What we want to do in the current case, is reading in the <code>u.data</code> file, and giving it proper headers:

In [0]:
all_rating_data = pandas.read_csv(
    movielens_paths['u.data'], # path to file to be read
    sep = '\t', # the data we are reading is tab-separated
    names = ['user_id', 'movie_id', 'rating', 'timestamp'] # custom indicated names for the columns to be read
)

In the way they are printed, dataframes may somewhat remind you of database tables:

In [0]:
all_rating_data.head()

Some further possibilities:

In [0]:
# return the data at row 3
print(all_rating_data.loc[3])

In [0]:
# return the rating of the data at row 3
print(all_rating_data.loc[3]['rating'])

In [0]:
# find all rows corresponding to ratings for Cinderella (movie ID 418)
cinderella_ratings = all_rating_data.loc[all_rating_data['movie_id'] == 418]

## Question
How many people rated Cinderella?

In [0]:
# enter your code here

## Question
Using the <code>cinderella_ratings</code> variable, extract and print the ratings of people who hated Cinderella (that is, who gave the movie a rating of 1).

In [0]:
# replace 'None' below by your code to do the proper extraction
cinderella_hate_ratings = None

# add your printing statement here

We might as well check the demographics characteristics of the people who gave Cinderella such a low score.

Demographics info is available in the file at location <code>movielens_paths['u.user']</code>.
As we stated before, this info is pipe-separated, a sample line is:

<code>14|45|M|scientist|55106</code>.

The values indicated correspond to the elements

<code>user_ID|age|gender|occupation|ZIP_code</code>.

## Question
Read the user demographics data into a variable <code>user_data</code>.

In [0]:
user_data = None # replace None by your code

# Uncomment the following line to display the demographics information of the Cinderella haters.
# user_data.loc[user_data['user_id'].isin(cinderella_hate_ratings['user_id'])]

## Selecting data of interest
We only want to keep ratings for the movies we are interested in. For this, we can make use of membership testing on our <code>movies_considered</code> variable:

In [0]:
ratings_for_movies = all_rating_data.loc[all_rating_data['movie_id'].isin(movies_considered)]

In [0]:
# how many ratings do we have for our considered movies?
len(ratings_for_movies)

pandas offers <code>unique()</code> and <code>nunique()</code> functions which are similar to <code>DISTINCT</code> and <code>COUNT(DISTINCT)</code> in SQL:

In [0]:
# how many unique users gave the ratings for our movies of interest?
print(ratings_for_movies['user_id'].nunique())

## Question
What is the sparsity proportion of the user-item matrix of the <code>ratings_for_movies</code> data?

That is, what is the ratio of zero-valued (so unrated) items over the total number of elements in the user-item matrix?

In [0]:
# replace the code below by your code to compute the sparsity proportion
3 / 4.0

## Starting small: focusing on actively rating users
As a first small example, let's first focus on a small set of users.

Let's say we want to select the 10 users who rated most of the movies of our interest.

Using <code>groupby</code> and <code>unique()</code>, we can flatten our rating table to collect a single list of rated movies per unique user ID:

In [0]:
users_to_rated_movies = ratings_for_movies.groupby('user_id')['movie_id'].unique()
# which movie IDs were rated by user 1?
users_to_rated_movies[1]

But we can also directly call <code>nunique()</code> to find the number of rated movies per user:

In [0]:
users_to_rating_count = ratings_for_movies.groupby('user_id')['movie_id'].nunique()
# how many movies were rated by user 1?
users_to_rating_count[1]

To identify the most active users, we sort the user IDs based on the amount of movies they rated, in descending order.

In [0]:
users_sorted_by_rating_count = users_to_rating_count.sort_values(ascending = False)
print(users_sorted_by_rating_count.head())

We now select the 10 most active users.

In [0]:
users_considered = users_sorted_by_rating_count[0:10].keys()
# again, for convenience we sort our table
users_considered = sorted(users_considered)

# is user 1 among the most active users?
print(1 in users_considered)
# is user 13 among the most active users?
print(13 in users_considered)

In [0]:
# If you want to see the demographics of the selected users, uncomment the following line.
# user_data.loc[user_data['user_id'].isin(users_considered)]

## Preparing the matrix factorization
We now know what movies we are interested in, and what users we are interested in. Now all we need to do is establishing a user-item matrix, and applying factorization on it.

We use numpy for handling matrices.

In [0]:
import numpy as np

## Question
What are the dimensions of the user-item matrix, which we will call $\mathbf{R}$ (as it holds ratings)?

Enter the appropriate number of rows and columns below, so we can initialize an all-zero matrix with the proper size.

In [0]:
num_rows = 1000 # Replace 1000 by the desired number of rows
num_columns = 1000 # Replace 1000 by the desired number of columns

R = np.zeros((num_rows, num_columns))

There is one more thing to resolve. Our user IDs and movie IDs are numbers, but they do not reflect the desired coordinates in a user-item matrix.

That is, R[0,0] should reflect the rating of 'the first user' on 'the first movie'.

In [0]:
# What is the lowest user ID we have?
print(users_considered[0])
# What is the lowest movie ID we have?
print(movies_considered[0])
print('So, R[0,0] should reflect the rating of user %d on movie %d.' % (users_considered[0], movies_considered[0]))

We therefore will create lookup maps, which map user and movie IDs to a matrix row/column index [0, 1, 2...], and the other way around.

In [0]:
user_to_index = dict(zip(users_considered, range(len(users_considered))))
movie_to_index = dict(zip(movies_considered, range(len(movies_considered))))
index_to_user = dict(zip(user_to_index.values(), user_to_index.keys()))      
index_to_movie = dict(zip(movie_to_index.values(), movie_to_index.keys()))

# feel free to print them to verify that the mapping worked out correctly.
# Note that the keys() of a dictionary are not necessarily returned in order.

At last, we now will populate the user-item matrix.

We do this by iterating over the relevant data frame rows, and then extracting the movie ID and rating score in case we encounter a user of interest.

In [0]:
for index, row in ratings_for_movies.iterrows():
    if row['user_id'] in users_considered:
        i = user_to_index[row['user_id']]
        j = movie_to_index[row['movie_id']]
        R[i, j] = row['rating']

In [0]:
# What is the user-item matrix looking like?
print(R)

For applying the matrix factorization, we use the <code>svd()</code> function in <code>scipy.linalg</code> here.

*Note that there are a few Singular Value Decomposition implementations offered by scipy and scikits-learn, also with dedicated approaches targeted at sparse matrices. But for our current smaller samples, the regular SVD works fine.*

In [0]:
import scipy.linalg

In [0]:
# How to call the SVD function?
help(scipy.linalg.svd)

We now perform the SVD. Then, we can express $\mathbf{R}$ as

$$\mathbf{R} = \mathbf{U} \mathbf{\Sigma} \mathbf{V^T}$$

with $\mathbf{U}$ being an orthonormal matrix expressing user-related factors,
$\mathbf{\Sigma}$ being a diagonal matrix expressing singular values of decreasing weight,
and $\mathbf{V}$ being an orthonormal matrix expressing movie-related factors,


In [0]:
U, S, Vt = scipy.linalg.svd(R, full_matrices=False)

In [0]:
print(U.shape)
print(S.shape)
print(Vt.shape)

In [0]:
# What are the first 10 singular values?
print(S[0:10])

## Plotting movies in latent factor space
Based on our analysis, we will plot the movies in latent factor space to see to what extent they cluster or contrast.

For the plotting, we will use matplotlib:

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
# add the 'magic' function above to ensure plots are rendered within the notebook
# for this, you either can use %matplot inline (which we do here), which will plot inline
# or % matplotlib notebook (which we did in the past lab), which will give some additional plot interaction possibilities

## Make 2D scatterplots of factor weights
We select the indices corresponding to the factors we want to examine.

Let's first take the first two factors, as they have the strongest weight.

In [0]:
factor_x_index = 0
factor_y_index = 1

Then, we can use the <code>scatter</code> function to draw a scatter plot.

In [0]:
# Recall that Vt is holding the movie-related factors
plt.scatter(Vt[factor_x_index,:], Vt[factor_y_index,:])
for movie_index in range(len(movies_considered)):
    plt.annotate(
        movie_data[index_to_movie[movie_index]],
        (Vt[factor_x_index, movie_index], Vt[factor_y_index, movie_index]))
plt.xlabel('factor %d' % (factor_x_index + 1))
plt.ylabel('factor %d' % (factor_y_index + 1))
# optional: save higher-resolution *.png locally
# plt.savefig('name_of_plot', ext='png', dpi=150)
plt.show()

## Rating prediction using matrix factorization
We can use matrix factorization to predict user ratings.

Looking at our 10 users, let's remove one from the user-item matrix, and try to predict the ratings for this user.

Say we remove the fourth user from <code>users_considered</code>:

In [0]:
user_to_predict = users_considered[3]
ratings_to_predict = R[3,:]
print('to predict: %s for user %s.' % (ratings_to_predict, user_to_predict))

We remove this user from R and re-establish the factorization on the new matrix.

In [0]:
# remove the user from users_considered
users_considered.remove(users_considered[3])
# redo the lookup indices
user_to_index = dict(zip(users_considered, range(len(users_considered))))
index_to_user = dict(zip(user_to_index.values(), user_to_index.keys()))      

# remove user from R
R = np.delete(R, (3), axis=0)
R

In [0]:
# redo the SVD
U, S, Vt = scipy.linalg.svd(R, full_matrices=False)

In [0]:
# make a new scatter plot
plt.figure()
plt.scatter(Vt[factor_x_index,:], Vt[factor_y_index,:])
for movie_index in range(len(movies_considered)):
    plt.annotate(
        movie_data[index_to_movie[movie_index]],
        (Vt[factor_x_index, movie_index], Vt[factor_y_index, movie_index]))
plt.xlabel('factor %d' % (factor_x_index + 1))
plt.ylabel('factor %d' % (factor_y_index + 1))
plt.show()

Note how this plot differs from your previous scatter plot.

The SVD solving approach suffers from sign indeterminacy, which means that the sign corresponding to a factor may flip for different initializations. Still, internal relationships should reasonably hold, and that indeed is the case (e.g. on factor 2, Psycho and Evita are still strong opposites).

## Performing rating prediction
Say we will only consider the first two factors, and try to predict the ratings using this model for our user 222.

We will first project the user's rating into 2D factor space, by multiplying with the columns of $\mathbf{V}$ corresponding to the two strongest factors:

In [0]:
U_S_2d = np.dot(ratings_to_predict, Vt[0:2,:].transpose())

Then, reconstruct the data by multiplying with the rows of $\mathbf{V^T}$ corresponding to the two strongest factors:

In [0]:
ratings_to_predict_reconstructed = np.dot(U_S_2d, Vt[0:2,:])
print(ratings_to_predict_reconstructed)
print(ratings_to_predict)

To get a better feel for how accurate the prediction is, implement the RMSE measure:

In [0]:
def RMSE(array1, array2):
    # Replace -1 by a proper RMSE (Root Mean Square Error) implementation.
    # You will at least need np.sqrt() -- consult the numpy documentation and use the IPython tab completion to further
    # establish the necessary functionality.
    return -1

# what is the RMSE for our example?
RMSE(ratings_to_predict, ratings_to_predict_reconstructed)

## I don't see how this SVD reconstruction works?
If you need to read up on your SVD background, check http://infolab.stanford.edu/~ullman/mmds/ch11.pdf and the corresponding examples.

## Wrapping up this lab assignment
We now gave you examples of how to perform matrix factorization on a movie ratings dataset.

For your lab course deliverable of this week, please play around with these examples more, and examine at least the effect of using different sizes of user populations.

* Our current example only used 10 (or 9) users, what would happen if you use slightly more, or many more? How are the main latent factors affected, do they become more stable as more users are considered? How is RMSE affected?

* In the scatter plot, play around with different factors. Instead of the first two, you can for example also test the first and the third.

* Optional: What would happen to the factorization if you would include other movies? Some nice ones to try are:
<code>
movieID 127: 'Godfather, The (1972)'
movieID 423: 'E.T. the Extra-Terrestrial (1982)'
movieID 485: 'My Fair Lady (1964)'
</code>
but feel free to consult <code>u.item</code> to select the IDs and titles of movies you find interesting.

As indicated above, hand in a \*.pdf report named<code>[studentNumberMember1_studentNumberMember2.pdf]</code> in which you include **three** latent factor scatter plots, obtained for **three different input data slices**, which at least differ in terms of user population size (so the size of <code>users_considered</code>.

For each of the scatter plots, do two things:
  * specify the characteristics of your input data (e.g. population size, selection criteria for users and films);
  * based on each plot, give a possible interpretation of what the factors represented by the x and y axes of your plot may indicate.

You can simply retrieve the scatter plot images for your report by using the 'download' button underneath each plot.

Alternatively, you can export higher-resolution images through <code>savefig</code>, e.g.

<code>$ plt.savefig('name_of_plot', ext='png', dpi=150)</code>.

Place this call before calling <code>plt.show()</code>; also see the commented line accompanying the first scatter plot.
